# 実践編テキスト：Deep Learning入門
### 1. ニューラルネットワークの基礎
ニューラルネットワークは層（レイヤー）で構成され、データを入力して重みを更新しながら出力を生成します。典型的な手順は以下の通りです。

- モデルの定義

- データの準備

- 損失関数と最適化アルゴリズムの設定

- 学習ループ（フォワード計算、損失計算、バックプロパゲーション）

### 2. PyTorchを使ったニューラルネットワークの基本構築
以下のコードで、シンプルな線形分類器を実装します。

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

# 1. モデルの定義
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc = nn.Linear(2, 1)  # 入力2次元、出力1次元
        
    def forward(self, x):
        return torch.sigmoid(self.fc(x))  # シグモイド関数を使用

# モデルインスタンスを作成
model = SimpleNN()

# 2. データの準備
# ANDゲートのデータ
x_data = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32)
y_data = torch.tensor([[0], [0], [0], [1]], dtype=torch.float32)

# 3. 損失関数と最適化アルゴリズム
criterion = nn.BCELoss()  # バイナリクロスエントロピー損失
optimizer = optim.SGD(model.parameters(), lr=0.1)

# 4. 学習ループ
for epoch in range(1000):
    # フォワード計算
    y_pred = model(x_data)
    loss = criterion(y_pred, y_data)
    
    # 勾配をリセットしてバックプロパゲーション
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/1000], Loss: {loss.item():.4f}')

# 結果の確認
with torch.no_grad():
    test_output = model(x_data)
    print("\nTest Results:")
    print(test_output.round())


Epoch [100/1000], Loss: 0.4354
Epoch [200/1000], Loss: 0.3463
Epoch [300/1000], Loss: 0.2901
Epoch [400/1000], Loss: 0.2509
Epoch [500/1000], Loss: 0.2216
Epoch [600/1000], Loss: 0.1986
Epoch [700/1000], Loss: 0.1801
Epoch [800/1000], Loss: 0.1648
Epoch [900/1000], Loss: 0.1518
Epoch [1000/1000], Loss: 0.1407

Test Results:
tensor([[0.],
        [0.],
        [0.],
        [1.]])


### 3. 実際の課題：MNISTデータセットでの手書き数字認識
MNISTは28x28ピクセルの手書き数字画像のデータセットで、10クラス（0～9）の分類を行います。

基本的な手順:

- データのロードと前処理。
- ネットワークの構築。
- モデルのトレーニング。
- 評価と可視化。

### 例題1: MNIST手書き数字認識の簡単なモデル
以下に、MNISTデータを使ったシンプルなMLP（多層パーセプトロン）モデルを記述します。

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# 1. データのロードと前処理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # 平均0.5, 標準偏差0.5で正規化
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# 2. モデルの定義
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)  # 入力は28x28ピクセル
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)  # 出力は10クラス
        
    def forward(self, x):
        x = x.view(-1, 28*28)  # Flatten
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = MLP()

# 3. 損失関数と最適化
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. トレーニングループ
for epoch in range(5):  # 5エポックだけ実行
    for images, labels in train_loader:
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch + 1}/5], Loss: {loss.item():.4f}')

# 5. テストと精度確認
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test data: {100 * correct / total:.2f}%')


Using downloaded and verified file: ./data\MNIST\raw\train-images-idx3-ubyte.gz
Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
<urlopen error [WinError 10060] 接続済みの呼び出し先が一定の時間を過ぎても正しく応答しなかったため、接続できませんでした。または接続済みのホストが応答しなかったため、確立された接続は失敗しました。>



100.0%


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
<urlopen error [WinError 10060] 接続済みの呼び出し先が一定の時間を過ぎても正しく応答しなかったため、接続できませんでした。または接続済みのホストが応答しなかったため、確立された接続は失敗しました。>



100.0%


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
<urlopen error [WinError 10060] 接続済みの呼び出し先が一定の時間を過ぎても正しく応答しなかったため、接続できませんでした。または接続済みのホストが応答しなかったため、確立された接続は失敗しました。>



100.0%


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw

Epoch [1/5], Loss: 0.2499
Epoch [2/5], Loss: 0.2842
Epoch [3/5], Loss: 0.1995
Epoch [4/5], Loss: 0.0270
Epoch [5/5], Loss: 0.2703
Accuracy on test data: 96.21%


この例題では、モデル構築、トレーニング、評価までを一通り学べます。さらに理解を深めるために、以下の改良を課題にすると良いです：

- ハイパーパラメータ（学習率、バッチサイズ）の変更。
- 層の数やニューロン数を増減させる。
- 画像のデータ拡張を試す。